In [1]:
import csv as csv 
import numpy as np

# Open up the csv file in to a Python object
data_all = []
with open('motEmo.csv',encoding="utf8") as train_file:
    csv_reader = csv.reader(train_file, delimiter=',', quotechar='"')
    for row in csv_reader:
        data_all.append(row)
data_all = np.array(data_all)
data = data_all[1::]

print(data)

[['1;à l’aise;1'] ['2;absorbé;1'] ['3;adouci;1'] ..., ['1332;prudent;10']
 ['1333;quelconque;10'] ['1334;terne;10']]


In [3]:
import pandas as pd
#importation des fichiers CSV
dfMotEmo = pd.read_csv('motEmo.csv',encoding="utf-8-sig",delimiter=";",header=0)
dfMotEmoAncien = pd.read_csv('motEmo(initial).csv',encoding="utf-8-sig",delimiter=";",header=0)
dfEmoCat = pd.read_csv('emoCat.csv',encoding="utf-8-sig",delimiter=";",header=0)

In [4]:
dfEmoCat

,emotion,categorie
0,tranquilité,1
1,Surprise,2
2,Joie,3
3,Tristesse,4
4,Dégoût,5
5,Colère,6
6,Fureur,7
7,Peur,8
8,Terreur,9
9,Coupure avec ses émotions,10


In [5]:
dfMotEmo[dfMotEmo['expression']=='énervé']['categorie']

571    6.0
Name: categorie, dtype: float64

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import nltk
from sklearn import linear_model, datasets

logreg = linear_model.LogisticRegression(C=1e5)

data_X_train=dfMotEmo['id'].to_frame()
data_y_train=dfMotEmo['categorie'].to_frame()

#data_X_train=dfEmoCat['categorie'].to_frame()
#data_y_train=dfEmoCat['emotion'].to_frame()

#logreg.fit(data_X_train, data_y_train)


#autre test de mahine learning
table= dfMotEmo[['expression','categorie']].as_matrix()
labeled_names = ([({'nom': name[0]}, name[1]) for name in table])
import random
random.shuffle(labeled_names)

classifier = nltk.NaiveBayesClassifier.train(labeled_names)

In [6]:
print(logreg.predict(518))

NotFittedError: This LogisticRegression instance is not fitted yet

In [7]:
phrasetest1="je suis énervées par tous ces papiers, il me mets dans tous ses états, joie"
phrasetest2="Vous avez tord de croire que la joie de vivre tient principalement aux rapports humains, vous vous trompez. Dieu en a mis dans tout ce qui nous entoure, on en trouve dans chaque détail, chaque petite chose de la vie quotidienne. Pour percevoir ces choses là, il suffirait de changer de point de vue. "
index=phrasetest1.find('par')
print(index)
phrasetest1.split()


17


['je',
 'suis',
 'énervées',
 'par',
 'tous',
 'ces',
 'papiers,',
 'il',
 'me',
 'mets',
 'dans',
 'tous',
 'ses',
 'états,',
 'joie']

In [8]:
from nltk.stem.snowball import FrenchStemmer
stemmer = FrenchStemmer()
stemmer.stem('ses')

'se'

In [9]:
from nltk.stem.snowball import FrenchStemmer
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer

#stemming pour du francais
stemmer = FrenchStemmer()
french_stopwords = set(stopwords.words('french'))
tokenizer = WordPunctTokenizer()

#nombre de ligne dans la table dfMotEmo
compteur=len(dfMotEmo.index)

rest=True;
while(rest):
    print('tapez 1 pour lancer le test')
    print('tapez 2 pour ajouter un mot ou une expression associé à une émotion à la base de donnée')
    print('tapez "q" pour quitter l\'application')
    inp=input('tapez la commande:')
    
    if inp=='1':
        
        texte=input("Entrez la texte à tester : ");
        #texte=phrasetest1
        res=[]
        #split=phrasetest1.split()
        split=texte.split()
        for mot in split:
            #nettoyage des données: j'enlève les ponctuations
            ponctuation=['.',',',';',':','!','?',')','(','\'','"']
            for p in ponctuation:
                if mot[0]==p:
                    mot=mot[1:]
                if mot[-1]==p:
                    mot=mot[:-1]
            #nettoyage des données: j'enleve les genres sur un mot pour qu'il soit reconnu dans la BDD
            if mot[-3:]=="ées" or mot[-3:]=="és" :
                mot=mot[:-3]+"é"
            if mot[-1:]=="s":
                mot=mot[:-1]
            #recherche dans la table des expression par emotion
            for expr in dfMotEmo['expression']:
                if stemmer.stem(expr)==stemmer.stem(mot):
                    id=dfMotEmo[dfMotEmo['expression']==expr]['id']
                    res.append([expr,id])
        #si c'est une expression de plusieurs mots
        textLessPron=tokenizer.tokenize(texte)
        textLessPron = [token for token in textLessPron if token.lower() not in french_stopwords]
        for expr in dfMotEmo['expression']:
            if len(expr.split()) > 1:
                #on enleve les pronoms
                exprLessPron=tokenizer.tokenize(expr)
                exprLessPron = [token for token in exprLessPron if token.lower() not in french_stopwords] 
                #on reconstruit le texte et l'expression nettoyés
                str1 = ' '.join(textLessPron)
                str2 = ' '.join(exprLessPron)
                #if texte.find(expr) != -1:
                if str1.find(str2) != -1:
                    print(expr)
                    cat=dfMotEmo[dfMotEmo['expression']==expr]['id']
                    res.append([expr,id])
                
        print(res)
        #prediction pour chaque mot et expression trouvés
        tabResultat=[]
        for r in res:
            #cat=logreg.predict(r[1])
            cat=classifier.classify({'nom':r[0]})
            print(cat)
            print(dfEmoCat[dfEmoCat['categorie']==cat]['emotion'])
            tabResultat.append(cat)
        #compte le nombre d'occurence pour chaque émotions trouvées
        print('table des id des émotions trouvées: ',tabResultat)
        compte = {}.fromkeys(set(tabResultat),0)
        for valeur in tabResultat:
            compte[valeur] += 1
        #prends l'occurence la plus haute
        n1=0
        for valeur in tabResultat:
            if compte[valeur] > n1:
                resu=valeur
                n1=compte[valeur]
        print('id de l\'emotion retenu (la plus grande occurence) :', resu)
        resu=dfEmoCat[dfEmoCat['categorie']==resu]['emotion'].values
        print('---------------RESULTAT----------------')
        print(resu[0])
        print('---------------RESULTAT----------------')
        print('')
        print('')
        
    if inp=='2':
            compteur=compteur+1
            exprAsave=input("Entrez l'expression ou le mot : ");
            print('table des émotions:')
            print('tranquilité = 1, Surprise=2, Joie=3, Tristesse=4, Dégoût=5, Colère=6, Fureur=7, Peur=8, Terreur=9, Coupure avec ses émotions=10')
            catAsave=input("Entrez le numéro de l\'émotion : ");
            fields=[compteur,exprAsave,catAsave]
            #ouverture du fichier
            with open(r'motEmo.csv', 'a',newline="\n") as f:
                writer = csv.writer(f,delimiter=';')
                #ajout d'une nouvelle ligne
                writer.writerow(fields)
            print('')
            print('')
            
            
    if inp=='q':
        rest=False
        print('fin du programme')

tapez 1 pour lancer le test
tapez 2 pour ajouter un mot ou une expression associé à une émotion à la base de donnée
tapez "q" pour quitter l'application
tapez la commande:q
fin du programme


In [71]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(data_X_train, data_y_train) 

C:\Users\Romain\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  app.launch_new_instance()


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [72]:
knn.predict(897)

array([10], dtype=int64)

In [73]:
rest=True;
while(rest):
    inp=input('taper la commande:')
    if inp=='1':
        rest=False


taper la commande:1


In [18]:
mot=',mot;'
mot[:-1]

',mot'

In [52]:

from nltk.corpus import names

table= dfMotEmo[['expression','categorie']].as_matrix()
labeled_names = ([({'nom': name[0]}, name[1]) for name in table])
import random
random.shuffle(labeled_names)



#featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
#train_set, test_set = featuresets[500:], featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(labeled_names)

In [60]:
classifier.classify({'nom':'traumatisé'})

4

In [21]:
cat=classifier.classify({'nom':'terreur'})
print(cat)
resu=dfEmoCat[dfEmoCat['categorie']==cat]['emotion'].values
print(resu[0])

8.0
Peur


In [8]:
table= dfMotEmo[['expression','categorie']].as_matrix()
labeled_names = ([({'nom': name[0]}, name[1]) for name in table])
import random
random.shuffle(labeled_names)

classifier = nltk.NaiveBayesClassifier.train(labeled_names)
print(nltk.classify.accuracy(classifier, labeled_names))

0.8148425787106447


In [9]:
table= dfMotEmoAncien[['expression','categorie']].as_matrix()
labeled_names = ([({'nom': name[0]}, name[1]) for name in table])
import random
random.shuffle(labeled_names)

classifier = nltk.NaiveBayesClassifier.train(labeled_names)
print(nltk.classify.accuracy(classifier, labeled_names))

0.8197088465845465
